## Dataset Construction
The following Notebook contains the code used for constructing our final dataset. This dataset is a combonation of the WA_TREE and WA_PLOT dataset accessed via the USFS FIA Datamart. The following sections will outline the data combonation algorithm and sanity checks done on the data to make sure it's fully valid and properly copied. We are aiming to keep the LAT, LON, MEASYEAR, MEASMON, MEASDAY from the WA_PLOT dataset in order to understand where our trees/forests are located and how they've been impacted by fires. 

### Algorithm 
We plan to combine the datasets along the column axes 'CN' from the WA_PLOT data and 'PLT_CN' from WA_TREE's since these are the two unique identifiers for each plot. We do this by using `pd.merge()` with WA_PLOT on the left and WA_TREE on the right, via the `left_on` and `right_on` arguments for alignment. This will additionally copy over LAT & LON values to all PLT_CN values were CN has a a LAT & LON. This will result in some dulicates given that there are duplicate years. Further documentation on `pd.merge` can be found at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html 

### Sanity Checks 
After the merge, all variables except those in the merge will be dropped and some sanity checks will be performed. Once sanity checks are passed, mostly checking that all LAT and LON carried over, we'll drop all variables that are non-essantial from the main dataset, such as variables not realated to our study, location, or duplicates from the merge. A full list of these will be given when it's done in the nb.

### Saving Final Dataset
The final dataset, after all un-needed variables have been dropped will be exported to a .csv file that will be zipped and uploaded to the group Github for all to use and manipulate. 

**WARNING: Do not attempt to plot all 500000+ values of this final dataset as a map. It WILL crash VSCode. I have yet to figure out a way around it**

In [52]:
import pandas as pd
import numpy as np

In [53]:
trees = pd.read_csv('/Users/ellapalacios/Downloads/tree_proj/data/WA_TREE.csv')
plots = pd.read_csv('/Users/ellapalacios/Downloads/tree_proj/data/WA_PLOT.csv')

/var/folders/nk/f27c2wr95rxdlns7bpdhxv680000gn/T/ipykernel_52090/380277768.py:1: DtypeWarning: Columns (77,159) have mixed types. Specify dtype option on import or set low_memory=False.
  trees = pd.read_csv('/Users/ellapalacios/Downloads/tree_proj/data/WA_TREE.csv')


In [54]:
#reducing data from WA_PLOT to only what we need

plots = plots[['CN', 'PLOT', 'INVYR', 'COUNTYCD', 'LAT', 'LON', 'ELEV', 'MEASYEAR', 'MEASMON', 'MEASDAY']] #takes this down to 1.7MB file instead of 11.6. 


In [55]:
#sorting each dataframe before combonation 

plots = plots.sort_values(by = 'CN')
trees  = trees.sort_values(by = 'PLT_CN')


In [56]:
#merging to create new dataset

new_data = pd.merge(plots, trees, left_on= 'CN', right_on= 'PLT_CN')

In [57]:
#creating sanity check from just what we need to check from new_data

sanity_check_data = new_data[['CN_x', 'PLT_CN', 'PLOT_x', 'PLOT_y', 'COUNTYCD_x', 'COUNTYCD_y', 'INVYR_x', 'INVYR_y' ]]

In [58]:
sanity_check_data.head(-5) #look at the top and bottom 5 rows to check for matching values across the board. 

,CN_x,PLT_CN,PLOT_x,PLOT_y,COUNTYCD_x,COUNTYCD_y,INVYR_x,INVYR_y
0,21116,21116,3,3,9,9,2001,2001
1,21116,21116,3,3,9,9,2001,2001
2,21116,21116,3,3,9,9,2001,2001
3,21116,21116,3,3,9,9,2001,2001
4,21116,21116,3,3,9,9,2001,2001
...,...,...,...,...,...,...,...,...
504946,786780595290487,786780595290487,82332,82332,3,3,2021,2021
504947,786780595290487,786780595290487,82332,82332,3,3,2021,2021
504948,786780595290487,786780595290487,82332,82332,3,3,2021,2021
504949,786780595290487,786780595290487,82332,82332,3,3,2021,2021


In [59]:
#testing using a pandas.testing function
from pandas.testing import assert_frame_equal

frame1 = sanity_check_data[['CN_x',  'PLOT_x', 'COUNTYCD_x', 'INVYR_x']]
frame2 = sanity_check_data[[ 'PLT_CN',  'PLOT_y', 'COUNTYCD_y', 'INVYR_y' ]]

frame1 = frame1.rename(columns={'CN_x':'CN', 'PLOT_x': 'PLOT', 'COUNTYCD_x':'COUNTYCD', 'INVYR_x' :'INVYR'})
frame2 = frame2.rename(columns={'PLT_CN':'CN', 'PLOT_y': 'PLOT', 'COUNTYCD_y':'COUNTYCD', 'INVYR_y' :'INVYR'})

assert_frame_equal(frame1, frame2, check_dtype= False ) #note: for this not to return an error, the columns MUST have the same names in each dataframe. 

#this returning nothing means they do infact line up. Assertion == passed :) 


With the sanity check passed, we can move on to dropping un-needed features (columns) and saving the file as a.csv . 

In [60]:
#dropping unnecessary features 
final_data = new_data.drop(columns= ['PREV_ACTUALHT_FLD', 'PREV_HT_FLD' ])

List of dropped columns (with def) from `new_data`.
+ STATECD == state code.  
+ xx_NERS == Has to do with North East Research Station, does not apply to our region.  
+ xx_SRS == data from South Research Station, does not apply to our region.  
+ xx_NCRS == data from North Central Research Station, does not apply to our region.  
+ xx_RMRS == data from Rocky Mountian Research Station, does not apply to our region.  
+ VOLCSSND == Sound cubic-foot wood volume in the sawlog portion of a sawtimber tree.  
+ DRYBIO_SAWLOG == Dry biomass of wood in the sawlog portion of a sawtimber tree.  
+ ACTUALHT_CALC_CD == Actual height, calculated, code.  
+ CULL_BF_ROTTEN == Rotten/missing board-foot cull of the sawlog.  
+ CULL_BF_ROTTEN_CD == Rotten/missing board-foot cull of the sawlog code.  
+ CULL_BF_ROUGH == 
+ CULL_BF_ROUGH_CD ==
+ VOLBSGRS == Gross board-foot wood volume in the sawlog portion of a sawtimber tree (Scribner
Rule).  
+ VOLBSNET == 
+ CULTURALLY_KILLED_PNWRS ==
+ GST_PNWRS ==
+ VOLTSSND == 
+ VOLCFGRS_BARK ==
+ VOLCFGRS_TOP == 
+ VOLCFGRS_TOP_BARK == 
+ PREV_ACTUAL_HT_FLD ==
+ PREV_HT_FLD == 




In [61]:
pd.options.display.max_columns = None
final_data.head(-10)

,CN_x,PLOT_x,INVYR_x,COUNTYCD_x,LAT,LON,ELEV,MEASYEAR,MEASMON,MEASDAY,CN_y,PLT_CN,PREV_TRE_CN,INVYR_y,STATECD,UNITCD,COUNTYCD_y,PLOT_y,SUBP,TREE,CONDID,PREVCOND,STATUSCD,SPCD,SPGRPCD,DIA,DIAHTCD,HT,HTCD,ACTUALHT,TREECLCD,CR,CCLCD,TREEGRCD,AGENTCD,CULL,DAMLOC1,DAMTYP1,DAMSEV1,DAMLOC2,DAMTYP2,DAMSEV2,DECAYCD,STOCKING,WDLDSTEM,VOLCFNET,VOLCFGRS,VOLCSNET,VOLCSGRS,VOLBFNET,VOLBFGRS,VOLCFSND,DIACHECK,MORTYR,SALVCD,UNCRCD,CPOSCD,CLIGHTCD,CVIGORCD,CDENCD,CDIEBKCD,TRANSCD,TREEHISTCD,BHAGE,TOTAGE,CULLDEAD,CULLFORM,CULLMSTOP,CULLBF,CULLCF,BFSND,CFSND,SAWHT,BOLEHT,FORMCL,HTCALC,HRDWD_CLUMP_CD,SITREE,CREATED_DATE,MODIFIED_DATE,MORTCD,HTDMP,ROUGHCULL,MIST_CL_CD,CULL_FLD,RECONCILECD,PREVDIA,P2A_GRM_FLG,TREECLCD_NERS,TREECLCD_SRS,TREECLCD_NCRS,TREECLCD_RMRS,STANDING_DEAD_CD,PREV_STATUS_CD,PREV_WDLDSTEM,TPA_UNADJ,DRYBIO_BOLE,DRYBIO_STUMP,DRYBIO_BG,CARBON_AG,CARBON_BG,CYCLE,SUBCYCLE,BORED_CD_PNWRS,DAMLOC1_PNWRS,DAMLOC2_PNWRS,DIACHECK_PNWRS,DMG_AGENT1_CD_PNWRS,DMG_AGENT2_CD_PNWRS,DMG_AGENT3_CD_PNWRS,MIST_CL_CD_PNWRS,SEVERITY1_CD_PNWRS,SEVERITY1A_CD_PNWRS,SEVERITY1B_CD_PNWRS,SEVERITY2_CD_PNWRS,SEVERITY2A_CD_PNWRS,SEVERITY2B_CD_PNWRS,SEVERITY3_CD_PNWRS,UNKNOWN_DAMTYP1_PNWRS,UNKNOWN_DAMTYP2_PNWRS,PREV_PNTN_SRS,DISEASE_SRS,DIEBACK_SEVERITY_SRS,DAMAGE_AGENT_CD1,DAMAGE_AGENT_CD2,DAMAGE_AGENT_CD3,CENTROID_DIA,CENTROID_DIA_HT,CENTROID_DIA_HT_ACTUAL,UPPER_DIA,UPPER_DIA_HT,VOLCSSND,DRYBIO_SAWLOG,DAMAGE_AGENT_CD1_SRS,DAMAGE_AGENT_CD2_SRS,DAMAGE_AGENT_CD3_SRS,DRYBIO_AG,ACTUALHT_CALC,ACTUALHT_CALC_CD,CULL_BF_ROTTEN,CULL_BF_ROTTEN_CD,CULL_BF_ROUGH,CULL_BF_ROUGH_CD,PREVDIA_FLD,TREECLCD_31_NCRS,TREE_GRADE_NCRS,BOUGHS_AVAILABLE_NCRS,BOUGHS_HRVST_NCRS,TREECLCD_31_NERS,AGENTCD_NERS,BFSNDCD_NERS,AGECHKCD_RMRS,PREV_ACTUALHT_RMRS,PREV_AGECHKCD_RMRS,PREV_BHAGE_RMRS,PREV_HT_RMRS,PREV_TOTAGE_RMRS,PREV_TREECLCD_RMRS,RADAGECD_RMRS,RADGRW_RMRS,VOLBSGRS,VOLBSNET,SAPLING_FUSIFORM_SRS,EPIPHYTE_PNWRS,ROOT_HT_PNWRS,CAVITY_USE_PNWRS,CORE_LENGTH_PNWRS,CULTURALLY_KILLED_PNWRS,DIA_EST_PNWRS,GST_PNWRS,INC10YR_PNWRS,INC5YRHT_PNWRS,INC5YR_PNWRS,RING_COUNT_INNER_2INCHES_PNWRS,RING_COUNT_PNWRS,SNAG_DIS_CD_PNWRS,CONEPRESCD1,CONEPRESCD2,CONEPRESCD3,MASTCD,VOLTSGRS,VOLTSGRS_BARK,VOLTSSND,VOLTSSND_BARK,VOLCFGRS_STUMP,VOLCFGRS_STUMP_BARK,VOLCFSND_STUMP,VOLCFSND_STUMP_BARK,VOLCFGRS_BARK,VOLCFGRS_TOP,VOLCFGRS_TOP_BARK,VOLCFSND_BARK,VOLCFSND_TOP,VOLCFSND_TOP_BARK,VOLCFNET_BARK,VOLCSGRS_BARK,VOLCSSND_BARK,VOLCSNET_BARK,DRYBIO_STEM,DRYBIO_STEM_BARK,DRYBIO_STUMP_BARK,DRYBIO_BOLE_BARK,DRYBIO_BRANCH,DRYBIO_FOLIAGE,DRYBIO_SAWLOG_BARK
0,21116,3,2001,9,48.333954,-124.638580,689.0,2000,8,29,4868680,21116,NaN,2001,53,6,9,3,1,4868680,1,NaN,1,263.0,13,13.6,1.0,56.0,1.0,NaN,2.0,65.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7087,NaN,22.796000,22.796000,22.038000,22.038000,118.135002,118.135002,22.796000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-17 02:38:27,2024-09-17 18:06:31,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.076000,597.437568,34.028706,196.609337,438.705030,99.680934,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,865.295917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.400638,112.375162,NaN,NaN,NaN
1,21116,3,2001,9,48.333954,-124.638580,689.0,2000,8,29,4879268,21116,NaN,2001,53,6,9,3,3,4879268,1,NaN,2,263.0,13,68.0,NaN,173.0,4.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,585.292705,976.256113,NaN,NaN,NaN,NaN,585.292705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-17 02:39:23,2024-09-17 18:06:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.483999,15339.351213,387.020787,4778.867014,11324.672132,2518.462916,

In [62]:
#dropping unnecessary features
final_data = final_data.drop(columns = ['CN_x', 'PLOT_x', 'COUNTYCD_x', 'INVYR_x']) #dropping duplicate features
final_data = final_data.dropna(axis = 1, how = 'all') #drops columns that are completely empty. 

In [63]:
#renaming remaining columns from duplicate features
final_data = final_data.rename(columns={'PLOT_y':'PLOT', 'COUNTYCD_y':'COUNTYCD', 'INVYR_y':'INVRY', 'CN_y':'CN'})


In [64]:
final_data.info(verbose=True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504956 entries, 0 to 504955
Data columns (total 144 columns):
 #    Column                          Non-Null Count   Dtype  
---   ------                          --------------   -----  
 0    LAT                             504956 non-null  float64
 1    LON                             504956 non-null  float64
 2    ELEV                            504956 non-null  float64
 3    MEASYEAR                        504956 non-null  int64  
 4    MEASMON                         504956 non-null  int64  
 5    MEASDAY                         504956 non-null  int64  
 6    CN                              504956 non-null  int64  
 7    PLT_CN                          504956 non-null  int64  
 8    PREV_TRE_CN                     224226 non-null  float64
 9    INVRY                           504956 non-null  int64  
 10   STATECD                         504956 non-null  int64  
 11   UNITCD                          504956 non-null  int64  
 12   

In [66]:
#saving current dataset -- Make sure you do a run all after any further changes as of 1:01 pm CDT 10/31/24

# df.to_csv('folder/subfolder/out.csv')   --- from pandas docs

final_data.to_csv('/Users/ellapalacios/Downloads/tree_proj/data/final_data.csv')

